## Language Project - Test 1

This note book contains the tests that I ran to generate recipies using grocery reciepts (first example).

In [ ]:
# Mount google drive

from google.colab import drive
drive.mount('/content/gdrive')
nbdir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
# Log in to hugging face

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Install modules

!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Instal

## Extracting text from reicpts

The first step is to extract the text from the images of grocery recipts. I am doing this using Easy OCR, a python module for extracting text from image. This tends to recognize the original text better but adds a extra characters.

In [ ]:
# Install modules

! pip install easyocr

! pip install torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 20.4 MB/s eta 0:00:00


In [ ]:
# Set the language to english

import easyocr

reader = easyocr.Reader(['en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

The dataset that I am using contains images of grocery recipts both with and without bounding boxes around the text. The version with teh bounding boxes performs better at extracting thext from the images.

In [ ]:
# Results using images without bounding boxes
result = reader.readtext("images/1.jpg")

In [ ]:
res = []

for (bbox, text, prob) in result:
    res.append(text)


food_res = " ".join(res)

print(food_res)

TRADER JOE'$ 2001 Greenvi Ile_Ave Da| as TX 75206 Store #403 (469) 334-0614 OPEN 8:OOAM TO 9:OOPM DAILY R-CARROTS   SHREDDED 10 0Z ERCUGDESEERUSFESTAU SAEE 8 TOMATOES   CRUSHED TOMAToES WHoLe_No S4LTOW/BASIL ORGANIC" olo_EASHIONED OATMEAL 69 PG S#REDDEDOMGEqESELLA Lite ; EGesshREO_edeGARIcREROHN! 8 8 BEANS ' GARBANZQ #proucebos #ssyLEag 4ct 0 A-APPLE BAG JAZZ 2 LB A-PEPPER BELL  EACH_XL RED 98 GROCERY NON  TAXABLE 0,49 0,87 BANANAS' ORGANIC 3F4 Eo29EENut Butter 2.49 CREAMY SALTED 1.69 WHL' WHT  PITA_BREAD  ,38 GROCERY NON TAXABLE @ 0.69 838.68 SUBTOTAL 838.68 TOTAL 840. 00 CASH 81,32 CHANGE ins Ryan ITEMS 22 12:34PM 0403 04 4398" 4683 06-28-2014 THANK YOU FOR SHOPPING AT TRADER JOE'$ WWw traderjoes.COM


In [ ]:
# Results using images with bounding boxes

result = reader.readtext("boxes/1.png")

In [ ]:
res = []

for (bbox, text, prob) in result:
    res.append(text)


food_res = " ".join(res)

print(food_res)



IRADER OES 2001 Greenvi Ile_Ave Da| as TX 75206 Store #403 (469) ` 334-0614 OPEN 8:OOAM TO 9:OOPM DAILY R-CARRUTS  SHREDDED 10 0z '29 R-CUCUMBERS PERSIAN LB 99 TOMATOES CRUSHED  NQ_SALT L= 55 TOMATOES WHOLE_No.SALT W/BASIL 1,.59 ORGANIC" oLd_FASHIONED OATMEAL 2.69 PEARL   TOMATOES . 2.49 PKG' SHREDDED MOZZARELLA LITE 99 EGGS DUL ORGANIC BROWN . 0,85 BEANS ' GARBANZO 2.99 SPROUTED CA_STYLE 3.99 A-AvoCADOS HASS_BAG 4CL A-APPLE BAG_JAZZ 2 LB 2,991 A-PEPPER BEL EACH_XLRFD 0,.99 GROCERY NON  TAXABLE 0,98 2 @ 0,.49 BANANAS  ORGANIC 3F4 0,29/E4 CREANY SAL TED PEANUT_BUTTER 2.49 MHL WH PITA_BREAD 38 GROCERY NON" TAXABLE 2 @ 0,69 838.68 SUBTOTAL 1382.68 DLA ~840 . 00 CASH 81,32 CHANGE ins Ryan ITEMS_22 12R4PR 0403 04 4398" 4683 06.28-2014 THANK YOU FOR SHOPPING AT TRADER JOE'$ Www traderjoes.coM MINI-F


## Identify food items from the text

The next step is to identify the food items from the text extracted from the recipt. To do this, I used a Named Entity Recognition (NER) model that was fine tuned to recognize one entity - FOOD.  

The first model that I tried for this was FoodBaseBERT, a fine-tuned BERT model that is ready to use for Named Entity Recognition of Food entities. Unfortunately this did not produce good results.

In [ ]:
# FOODBaseBERT

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Dizex/FoodBaseBERT")
model = AutoModelForTokenClassification.from_pretrained("Dizex/FoodBaseBERT")

pipe = pipeline("ner", model=model, tokenizer=tokenizer, device="cuda")

ner_entity_results = pipe(food_res)
print(ner_entity_results)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/20.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/814 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

[{'entity': 'B-FOOD', 'score': 0.7157023, 'index': 162, 'word': '##Z', 'start': 305, 'end': 306}, {'entity': 'B-FOOD', 'score': 0.6802275, 'index': 163, 'word': '##Z', 'start': 306, 'end': 307}, {'entity': 'B-FOOD', 'score': 0.60307306, 'index': 192, 'word': 'GA', 'start': 360, 'end': 362}, {'entity': 'B-FOOD', 'score': 0.6725315, 'index': 194, 'word': '##AN', 'start': 364, 'end': 366}, {'entity': 'B-FOOD', 'score': 0.64056814, 'index': 195, 'word': '##Z', 'start': 366, 'end': 367}, {'entity': 'B-FOOD', 'score': 0.7743431, 'index': 214, 'word': 'A', 'start': 399, 'end': 400}, {'entity': 'B-FOOD', 'score': 0.8598087, 'index': 215, 'word': '##vo', 'start': 400, 'end': 402}, {'entity': 'B-FOOD', 'score': 0.7754797, 'index': 216, 'word': '##CA', 'start': 402, 'end': 404}, {'entity': 'B-FOOD', 'score': 0.554307, 'index': 217, 'word': '##D', 'start': 404, 'end': 405}, {'entity': 'B-FOOD', 'score': 0.5491591, 'index': 235, 'word': 'J', 'start': 433, 'end': 434}, {'entity': 'B-FOOD', 'score': 

In [ ]:
for i in ner_entity_results:
  print(i["word"])

##Z
##Z
GA
##AN
##Z
A
##vo
##CA
##D
J
##Z
##Z
##NA


The other model that I tried was roberta-base-food-ner, which is a fine-tuned version of RoBERTa base on a dataset of about 300 hand-labeled recipes. I was able to get better results using this model, after converting the input text to lower case.

In [ ]:
# roberta-base-food-ner

model = AutoModelForTokenClassification.from_pretrained('carolanderson/roberta-base-food-ner')
tokenizer = AutoTokenizer.from_pretrained("roberta-base", add_prefix_space=True)
nlp = pipeline("ner", model=model, tokenizer=tokenizer, device="cuda")


config.json:   0%|          | 0.00/896 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
results = nlp(food_res.lower(), aggregation_strategy="first")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:392: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


In [ ]:
items = []
for i in results:
  print(i["word"])
  items.append(i["word"])

 tomatoes
 tomatoes
 oatmeal
 tomatoes
 mozzarella
 eggs
 beans
 garbanzo
 sprouted
 bag_jazz
 bananas
 creany
 peanut_butter
 pita_bread


## Generating recipies

The last step is to generate recipies using the the names of food items identified from the grocery recipt.

The first method that I used was a fintuned lora adapter on a dataset of recipies for the llama 3.1 8B Base model. This method works pretty well, however when there are a large number of items in the input list, the output of the model does not follow the format that it was trained to produce.

In [ ]:
# Load the adapter

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "yr82/recipie-lora-llama-2"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

In [ ]:
def format_list(items):
    formatted_list = ', '.join([f'"{item}"' for item in items])
    return f'[{formatted_list}]'

In [ ]:
# Remove duplicates and use only the first 5 items from the list

unique_food = list(dict.fromkeys(items))

unique_food = unique_food[:5]

food_list = format_list(unique_food)

In [ ]:
# Generate recipie

batch = tokenizer(food_list + " ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=512, min_new_tokens=64, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)

output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)


<ipython-input-23-38d3787dac53>:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1902: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
sections = output_text.split("\n")

for section in sections:
  print(section)


[" tomatoes", " oatmeal", " mozzarella", " eggs", " beans"] ->:  title: Baked Tomatoes
ingredients: ["1 can tomatoes", "1/2 c. oatmeal", "1/2 c. mozzarella cheese", "2 eggs", "1 can beans"]
directions: ["Mix all ingredients together.", "Pour into 9 x 13-inch pan.", "Bake at 350\u00b0 for 1 hour."]


The next method that I tried was using a language model that was fine-tuned specifically for recipie generting recipies. Chef Transformer is a fine-tuned version of the T5 language model trained on cooking recipies. This method works well too, however the directions generated do not always make sense.

In [ ]:
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer

MODEL_NAME_OR_PATH = "flax-community/t5-recipe-generation"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, use_fast=True)
model = FlaxAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)

prefix = "items: "

generation_kwargs = {
    "max_new_tokens": 512,
    "min_new_tokens": 64,
    "num_return_sequences": 1,
    "eos_token_id": tokenizer.eos_token_id
}


special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}
def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, "")

    return text

def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]

    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts

def generation_function(texts):
    _inputs = texts if isinstance(texts, list) else [texts]
    inputs = [prefix + inp for inp in _inputs]
    inputs = tokenizer(
        inputs,
        padding="longest",
        truncation=True,
        return_tensors="jax"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **generation_kwargs
    )
    generated = output_ids.sequences
    generated_recipe = target_postprocessing(
        tokenizer.batch_decode(generated, skip_special_tokens=False),
        special_tokens
    )
    return generated_recipe


tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

flax_model.msgpack:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
# Generate recipie

generated = generation_function(items)[0]

In [ ]:
sections = generated.split("\n")


for section in sections:
  print(section)



 title: roasted tomatoes
 ingredients: 1 can whole tomatoes
 ingredients: 1 jar roasted tomatoes
 directions: drain tomatoes.-- put in a baking dish.-- bake at 350 for 30 minutes.


In [ ]:
# Generate using only the first 5 items

generated = generation_function(unique_food)[0]

In [ ]:
sections = generated.split("\n")

for section in sections:
  print(section)

 title: roasted tomatoes
 ingredients: 1 can whole tomatoes
 ingredients: 1 jar roasted tomatoes
 directions: drain tomatoes.-- put in a baking dish.-- bake at 350 for 30 minutes.


The last method that I tried was using OpenAI's API to interface with GPT-4 to generate recipies. This method generates the best results out all the methods that I tried.

In [ ]:
! pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
def format_string(items):
    formatted_list = ', '.join([f'"{item}"' for item in items])
    return f'{formatted_list}'

In [ ]:
item_string = format_string(items)

In [ ]:
print(item_string)

" tomatoes", " tomatoes", " oatmeal", " tomatoes", " mozzarella", " eggs", " beans", " garbanzo", " sprouted", " bag_jazz", " bananas", " creany", " peanut_butter", " pita_bread"


In [ ]:
import os

os.environ["OPENAI_API_KEY"]  = "OPENAI_API_KEY"

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Give me a recipe using the following ingredients - " + item_string}
  ]
)

generated = completion.choices[0].message.content

In [ ]:
sections = generated.split("\n")

for section in sections:
  print(section)

Sure, I can help with that! Considering the ingredients you have listed, let's make a "Tomato and Mozzarella Pita Pocket with Creamy Peanut Butter Banana Oatmeal." It seems we have some duplicate ingredients (tomatoes listed multiple times), so I'll only count one of each type. Here’s a recipe that incorporates most of your ingredients:

**Tomato and Mozzarella Pita Pocket:**

**Ingredients:**
- 2-3 fresh tomatoes, diced
- 1 cup mozzarella cheese, shredded or in slices
- 1 pita bread, cut in half to form pockets
- 1 egg
- 1 cup sprouted beans or garbanzo beans
- Salt and pepper to taste
- Olive oil (optional)

**Instructions:**
1. **Prep the Ingredients:**
    - Dice the tomatoes and sprinkle with a little salt. Let them sit for a few minutes and then drain any excess liquid.
    - Rinse the sprouted beans or garbanzo beans (if canned, drain and rinse them).

2. **Cook the Egg:**
    - In a small skillet, fry the egg to your liking (sunny-side up, scrambled, etc.). Season with salt and